In [ ]:
import os
import cv2
import pickle
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.use('tkagg')


# Load the dataset
folder = 'c:/object-detection-crowdai'
frames = os.listdir(folder)

boxes = []

# Sort (alphabetically) to ensure temporal consecutiveness
frames.sort()
idx = frames.index('1479498995506866553.jpg')

# Let's assume the detector has detected a vehicle
x1, y1 = 965, 555
x2, y2 = 1015, 595


width = x2 - x1
height = y2 - y1

# Limit the search to a certain vicinity (since the cars can only move that fast)
search = 50

# Set up tracker
tracker_types = ['MIL','KCF', 'CSRT']
tracker_type = tracker_types[2]

if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()

if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create()

if tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()

# Genrate tracking template
img = cv2.imread(os.path.join(folder, frames[idx]))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Initialize tracker
bbox = (x1, y1, width, height)
ok = tracker.init(img, bbox)


# Tracking loop
for ii in range(idx, idx + 50):
    img = cv2.imread(os.path.join(folder, frames[ii]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
        
    ok, bbox = tracker.update(img)
    print(ok, bbox)

            
    # Show the tracker working
    x1, y1 = bbox[0], bbox[1]
    width, height = bbox[2], bbox[3]
    cv2.rectangle(img, (x1, y1), (x1+width, y1+height), (0, 255, 0), 2)
    plt.imshow(img)
    plt.show(), plt.draw()    
    plt.waitforbuttonpress(0.1)
    plt.clf()
    
    #boxes.append([x1, y1, width, height])
    # with open('/home/janko/Projects/robot_dreams/cv/data/tracking/csrt/boxes.pckl', 'wb') as fid:
    #     pickle.dump(boxes, fid)

### Step 6

### Comparing the results:

#### "Do you see any differences? If so, what are they?" + "Does one tracker perform better than the other? In what way?"

I tried  3 different tracker from your example - "MIL", "KCF", "CSRT".

MIL - This tracker well identified the object on the first frame and in fact kept the target in the frame quite well, but there are several problems:
- the size of the detection frame does not change dynamically depending on the distance to the captured object
- after the car disappears from the frame, the tracker still shows the capture frame from the last frame on which the object (car) was still detected

KCF - This track didn't detect the object in the video at all, idk why 

CSRT - As for me, he did the best job, because this tracker worked the same way as the first one, but unlike the first one, this tracker does not have the first problem with changing the size of the frame around the object, but only the second one.